In [1]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
!ls 'drive/My Drive'

In [ ]:
!pip install pretrainedmodels

     |████████████████████████████████| 61kB 3.8MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60964 sha256=e2277f3460b8aaa4ea1b48f729cbff21a4eca460d59d9f4704812948de24d395
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built pretrainedmodels


In [ ]:
!pip install nnAudio==0.1.4a0

In [ ]:
!pip install albumentations

     |████████████████████████████████| 634kB 8.6MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654021 sha256=0c45ce71a8d0b11b6d8b2d1ffeeaca391fc942f2c0418b091b459e760364e6d1
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [ ]:
!pip install torch-lr-finder


In [ ]:
import sys

sys.path.insert(0,'drive/My Drive/GIZ_COMP/')

import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import os 
import random

import librosa
import albumentations as A
from torch.utils.data import Dataset
from torch.utils.data import Dataset,WeightedRandomSampler,TensorDataset
import torch.nn as nn
import pretrainedmodels
from torch.nn import functional as F
from torch.cuda import amp
from scipy.io import wavfile

from PIL import Image
from PIL import ImageFile
from source.utils import GIZ_Dataset, Engine, resnet34, GIZ_Test_Dataset,wav_to_img
#import nlpaug.flow as naf
#import nlpaug.augmenter.spectrogram as nas

seed=1234


torch.backends.cudnn.benchmark = False



ImageFile.LOAD_TRUNCATED_IMAGES = True

def set_all_seeds(seed):
  random.seed(seed)
  #os.environ('PYTHONHASHSEED') = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True

set_all_seeds(1234)  

In [ ]:
'''import subprocess
import os

def convert_webm_to_wav(file,old_dir,new_dir):
    #ffmpeg -i ./big-buck-bunny_trailer.webm -c:a pcm_f32le ./out.wav
    command = ['ffmpeg', '-i', os.path.join(old_dir,file), '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '22050', os.path.join(new_dir,file)]
    subprocess.run(command,stdout=subprocess.PIPE,stdin=subprocess.PIPE)
    
path1 = 'drive/My Drive/GIZ_COMP/inputs/train_audio/'
path2 = 'drive/My Drive/GIZ_COMP/inputs/train_audio_wav/'
list_audio = os.listdir(path1)
for aud in tqdm(list_audio):
    convert_webm_to_wav(aud,path1,path2)
#print(max_train_length)  #60858
'''

"import subprocess\nimport os\n\ndef convert_webm_to_wav(file,old_dir,new_dir):\n    #ffmpeg -i ./big-buck-bunny_trailer.webm -c:a pcm_f32le ./out.wav\n    command = ['ffmpeg', '-i', os.path.join(old_dir,file), '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '22050', os.path.join(new_dir,file)]\n    subprocess.run(command,stdout=subprocess.PIPE,stdin=subprocess.PIPE)\n    \npath1 = 'drive/My Drive/GIZ_COMP/inputs/train_audio/'\npath2 = 'drive/My Drive/GIZ_COMP/inputs/train_audio_wav/'\nlist_audio = os.listdir(path1)\nfor aud in tqdm(list_audio):\n    convert_webm_to_wav(aud,path1,path2)\n#print(max_train_length)  #60858\n"

In [ ]:
save_path = 'drive/My Drive/GIZ_COMP/inputs/giz_train_images/'
path = 'drive/My Drive/GIZ_COMP/inputs/train_audio_wav/'
signal_length = None
wav_to_img(path,save_path,signal_length)

In [ ]:

DEVICE = "cuda"
EPOCHS = 40
batch_size = 16

def train_freq(fold,save_model=False):
     
    df = pd.read_csv('drive/My Drive/GIZ_COMP/inputs/stratified_train_5_fold_fnl1.csv')
    df_train = df[df.kfold!=fold].reset_index(drop=True)


    df_val = df[df.kfold==fold].reset_index(drop=True)


    x_train = df_train.loc[:,["fn"]]
    y_train = df_train.loc[:,["label"]].values

    print(x_train.shape)
    print(y_train.shape)
    x_val = df_val.loc[:,["fn"]]
    y_val = df_val.loc[:,["label"]].values
    
    print(x_val.shape)
    print(y_val.shape)
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)

    
    train_aug = A.Compose([
                 A.Normalize(mean,std,max_pixel_value=255.0,always_apply=True),
               A.CenterCrop(224,224),
               A.Cutout(num_holes=8,max_h_size=0,max_w_size=224,p=0.1)
                                                   
           ])
  
    
    val_aug = A.Compose([   
               A.Normalize(mean,std,max_pixel_value=255.0,always_apply=True),
               A.CenterCrop(224,224),                                                                          
           ])

    train_data= GIZ_Dataset('drive/My Drive/GIZ_COMP/inputs/giz_train_images',x_train, y_train,transform=train_aug)


    train_data_loader = torch.utils.data.DataLoader(train_data,batch_size=32,
    shuffle=True,num_workers=2,pin_memory=True)


    val_data= GIZ_Dataset('drive/My Drive/GIZ_COMP/inputs/giz_train_images',x_val, y_val,transform=val_aug)


    val_data_loader = torch.utils.data.DataLoader(val_data,batch_size=32,
    shuffle=False,num_workers=2,pin_memory=True)
    
    step_size = 2*(x_train.shape[0]//batch_size)
    model = resnet34(pretrained=True)
    model.to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
 
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=2,threshold=1e-6,mode='min',verbose=True)


    eng = Engine(model,optimizer,DEVICE)
    scaler = amp.GradScaler()
    
    best_loss = np.inf
    early_stoping = 10
    early_stoping_counter = 0

    for epoch in range(EPOCHS):
        train_loss = eng.train(train_data_loader,scaler)
        val_loss = eng.validate(val_data_loader)
        scheduler.step(val_loss)

        print(f"fold={fold}",f"epoch={epoch}",f"train_loss={train_loss}",f"val_loss={val_loss}")
        
        if val_loss<best_loss:
            best_loss= val_loss
            if save_model:
                torch.save(model.state_dict(),f"drive/My Drive/GIZ_COMP/models/audio1_model1_fold{fold}.bin")
                print(f"fold={fold}",f"best_val_loss={best_loss}")
                early_stoping_counter = 0
        else: 
            early_stoping_counter += 1
            print('count :', early_stoping_counter)

        if early_stoping_counter > early_stoping:
            break
    
       
    
for i in range(5):
    train_freq(fold=i,save_model=True)



(3767, 1)
(3767, 1)
(942, 1)
(942, 1)
fold=0 epoch=0 train_loss=4.734504297628241 val_loss=3.752889617284139
fold=0 best_val_loss=3.752889617284139
fold=0 epoch=1 train_loss=3.1600962634813987 val_loss=2.658652742703756
fold=0 best_val_loss=2.658652742703756
fold=0 epoch=2 train_loss=2.01382574590586 val_loss=1.9125886758168538
fold=0 best_val_loss=1.9125886758168538
fold=0 epoch=3 train_loss=1.3558237608206474 val_loss=1.5462702512741089
fold=0 best_val_loss=1.5462702512741089
fold=0 epoch=4 train_loss=1.0019230153095924 val_loss=1.5022492090861002
fold=0 best_val_loss=1.5022492090861002
fold=0 epoch=5 train_loss=0.7450131965895831 val_loss=1.3481016675631206
fold=0 best_val_loss=1.3481016675631206
fold=0 epoch=6 train_loss=0.5677954048170881 val_loss=1.2401958107948303
fold=0 best_val_loss=1.2401958107948303
fold=0 epoch=7 train_loss=0.46081635059188986 val_loss=1.2784447411696116
count : 1
fold=0 epoch=8 train_loss=0.39226469182867113 val_loss=1.2754830360412597
count : 2
fold=0 epo

Exception in thread Thread-357:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/pin_memory.py", line 25, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.6/dist-packages/torch/multiprocessing/reductions.py", line 282, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/usr/lib/python3.6/multiprocessing/reduction.py", line 182, in recv_handle
    return recvfds(s, 1)[0]
  File "/usr/lib/python3.6/multiprocessing/reduction.py", line 153, in recvfds
    msg, a

KeyboardInterrupt: ignored

In [ ]:

DEVICE = "cuda"
#model_path = f"drive/My Drive/GIZ_COMP/models/audio5_model1_fold{i}.bin"
test_path = 'drive/My Drive/GIZ_COMP/inputs/SampleSubmission_GIZ.csv'
test_img_path = 'drive/My Drive/GIZ_COMP/inputs/giz_test_images'
predictions = []
audio_ids = []
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
for i in range(5):

    model = resnet34(pretrained=False)
    model.load_state_dict(torch.load(f"drive/My Drive/GIZ_COMP/models/audio1_model1_fold{i}.bin"))
    model= model.to(DEVICE)
    model.eval()
    test_ids = pd.read_csv(test_path)
    test_ids['fn'] = test_ids['fn'].str.replace('audio_files/','')

    test_aug = A.Compose([
               A.Normalize(mean,std,max_pixel_value=255.0,always_apply=True),                                                                         
           ])
    test_data= GIZ_Test_Dataset(test_img_path,test_ids,transform=test_aug)

    test_data_loader = torch.utils.data.DataLoader(test_data,batch_size=16,
         shuffle=False,num_workers=8,pin_memory=True)
  
    with torch.no_grad():
      for idx,_data in enumerate(test_data_loader):
        audio = _data['x']
        audio_id = _data['aud']
        audio = audio.to(DEVICE,dtype=torch.float)
        #print(image.shape)
        pred = model(audio)
        pred = torch.softmax(pred,dim=1)
        pred = pred.detach().cpu().numpy()
        audio_ids.append(audio_id)
        predictions.append(pred)
      


In [ ]:
pred_0 = np.concatenate(predictions,axis=0)
print(pred_0.shape)
#print(np.argmax(pred_0,axis=1))
n_pred = pred_0.reshape(5,1017,193)
avg = np.mean(n_pred,axis=0)
print(avg.shape)

(5085, 193)
(1017, 193)


In [ ]:
#label mappings 

labels = ['Pump', 'Spinach', 'abalimi', 'afukirira', 'agriculture', 'akammwanyi', 'akamonde', 'akasaanyi', 'akatunda', 'akatungulu', 'akawuka', 'amakoola', 'amakungula', 'amalagala', 'amappapaali', 'amatooke', 'banana', 'beans', 'bibala', 'bulimi', 'butterfly', 'cabbages', 'cassava', 'caterpillar', 'caterpillars', 'coffee', 'crop', 'ddagala', 'dig', 'disease', 'doodo', 'drought', 'ebbugga', 'ebibala', 'ebigimusa', 'ebijanjaalo', 'ebijjanjalo', 'ebikajjo', 'ebikolo', 'ebikongoliro', 'ebikoola', 'ebimera', 'ebinyebwa', 'ebirime', 'ebisaanyi', 'ebisooli', 'ebisoolisooli', 'ebitooke', 'ebiwojjolo', 'ebiwuka', 'ebyobulimi', 'eddagala', 'eggobe', 'ejjobyo', 'ekibala', 'ekigimusa', 'ekijanjaalo', 'ekikajjo', 'ekikolo', 'ekikoola', 'ekimera', 'ekirime', 'ekirwadde', 'ekisaanyi', 'ekitooke', 'ekiwojjolo', 'ekyeya', 'emboga', 'emicungwa', 'emisiri', 'emiyembe', 'emmwanyi', 'endagala', 'endokwa', 'endwadde', 'enkota', 'ennima', 'ennimiro', 'ennyaanya', 'ensigo', 'ensiringanyi', 'ensujju', 'ensuku', 'ensukusa', 'enva endiirwa', 'eppapaali', 'faamu', 'farm', 'farmer', 'farming instructor', 'fertilizer', 'fruit', 'fruit picking', 'garden', 'greens', 'ground nuts', 'harvest', 'harvesting',
          'insect', 'insects', 'irish potatoes', 'irrigate', 'kaamulali', 'kasaanyi', 'kassooli', 'kikajjo', 'kikolo', 'kisaanyi', 'kukungula', 'leaf', 'leaves', 'lumonde', 'lusuku', 'maize', 'maize stalk borer', 'maize streak virus', 'mango', 'mangoes', 'matooke', 'matooke seedlings', 'medicine', 'miceere', 'micungwa', 'mpeke', 'muceere', 'mucungwa', 'mulimi', 'munyeera', 'muwogo', 'nakavundira', 'nambaale', 'namuginga', 'ndwadde', 'nfukirira', 'nnakati', 'nnasale beedi', 'nnimiro', 'nnyaanya', 'npk', 'nursery bed', 'obulimi', 'obulwadde', 'obumonde', 'obusaanyi', 'obutunda', 'obutungulu', 'obuwuka', 'okufukirira', 'okufuuyira', 'okugimusa', 'okukkoola', 'okukungula', 'okulima', 'okulimibwa', 'okunnoga', 'okusaasaana', 'okusaasaanya', 'okusiga', 'okusimba', 'okuzifuuyira', 'olusuku', 'omuceere', 'omucungwa', 'omulimi', 'omulimisa', 'omusiri', 'omuyembe', 'onion', 'orange', 'pampu', 'passion fruit', 'pawpaw', 'pepper', 'plant', 'plantation', 'ppaapaali', 'pumpkin', 'rice', 'seed', 
          'sikungula', 'sow', 'spray', 'spread', 'suckers', 'sugarcane', 'sukumawiki', 'super grow', 'sweet potatoes', 'tomatoes', 'vegetables', 'watermelon', 'weeding', 'worm']

print(labels[25]) 


coffee


In [ ]:
for i, label in enumerate(labels):
  test_ids[label] = [t[i] for t in pred_0]
test_ids.head() 
test_ids.to_csv('drive/My Drive/GIZ_COMP/inputs/sub.csv', index=False)
test_ids.head()

,fn,maize streak virus,disease,okukkoola,muwogo,mpeke,mucungwa,greens,garden,mango,bulimi,obuwuka,ebikoola,obulimi,ebisoolisooli,kaamulali,eddagala,beans,omuyembe,leaf,kisaanyi,leaves,butterfly,okuzifuuyira,micungwa,ppaapaali,emboga,kikolo,harvest,olusuku,coffee,super grow,rice,ensujju,okulima,worm,ebbugga,onion,ensigo,plantation,...,ejjobyo,omulimi,okusimba,sweet potatoes,okufuuyira,farming instructor,nnasale beedi,passion fruit,ekitooke,ebisaanyi,ekyeya,enva endiirwa,emisiri,emiyembe,amatooke,ebiwuka,farm,ebinyebwa,amappapaali,ebimera,kassooli,harvesting,emmwanyi,akamonde,obumonde,cabbages,akasaanyi,spread,ebirime,drought,kasaanyi,suckers,insects,fertilizer,nakavundira,ekiwojjolo,akawuka,ddagala,ebiwojjolo,obutungulu
0,00118N3.wav,0.001904,0.000555,0.000628,0.000492,0.000179,0.000970,0.002059,0.003454,0.000227,0.000111,0.008612,0.000211,0.000195,0.004386,0.000685,0.000634,0.001874,0.000264,0.001865,0.001436,0.000844,0.035215,0.002442,0.000706,0.003954,0.000457,0.000338,0.003942,0.000115,0.021425,0.000280,0.011231,0.000840,0.000032,0.000273,0.000160,0.000135,0.000332,0.001648,...,0.000104,0.000039,0.002311,0.000460,0.005301,0.002158,0.000727,0.001118,0.005031,0.004282,0.000431,0.000075,0.000425,0.000335,0.000529,0.002397,0.000145,0.002615,0.003811,0.000311,0.000677,0.000666,0.000083,0.001102,0.000435,0.001982,0.098408,0.000242,0.000079,0.002136,0.003859,0.283925,0.000980,0.006723,0.005748,0.000190,0.034194,0.005212,0.000295,0.000683
1,00P0NMV.wav,0.000114,0.000036,0.000092,0.000340,0.000062,0.000641,0.000079,0.000340,0.000052,0.000276,0.000354,0.000152,0.000405,0.000221,0.000187,0.000215,0.000045,0.000066,0.000069,0.000187,0.000031,0.000110,0.000240,0.000268,0.000258,0.000354,0.000072,0.000071,0.000025,0.000042,0.000066,0.000037,0.000593,0.000116,0.000191,0.001137,0.000056,0.000118,0.000966,...,0.000081,0.000177,0.000326,0.000198,0.000261,0.000388,0.001220,0.000448,0.000084,0.000269,0.000038,0.000204,0.000160,0.000060,0.000288,0.000327,0.000021,0.000053,0.000424,0.000149,0.000085,0.000072,0.000269,0.000304,0.000298,0.000136,0.000200,0.000040,0.000052,0.000039,0.000560,0.000165,0.000093,0.000353,0.909589,0.001492,0.000653,0.000866,0.000317,0.000642
2,01QEEZI.wav,0.000061,0.003933,0.000982,0.008510,0.000258,0.000217,0.000065,0.000440,0.000054,0.192283,0.000104,0.000099,0.007933,0.000068,0.001013,0.000083,0.000201,0.071119,0.001991,0.000108,0.002438,0.000226,0.000079,0.000153,0.000581,0.000849,0.000124,0.000105,0.000421,0.000293,0.000046,0.000147,0.000075,0.000205,0.001080,0.000858,0.000107,0.000360,0.001083,...,0.001004,0.003934,0.000139,0.000006,0.000227,0.000216,0.000143,0.000041,0.000163,0.000094,0.000111,0.003327,0.000966,0.032828,0.000262,0.000209,0.000677,0.006549,0.000575,0.031873,0.000203,0.000059,0.002329,0.000121,0.001207,0.000045,0.000188,0.000097,0.429000,0.004770,0.000208,0.000086,0.000050,0.000396,0.000089,0.000128,0.000208,0.000186,0.000471,0.001070
3,037YAED.wav,0.000500,0.000516,0.000089,0.000221,0.008674,0.000172,0.000235,0.236752,0.000076,0.002010,0.000478,0.000140,0.006835,0.000518,0.000697,0.000239,0.000247,0.001286,0.001730,0.000208,0.001137,0.001939,0.000069,0.000225,0.000823,0.000273,0.000461,0.001124,0.017625,0.001720,0.000267,0.000230,0.000873,0.000201,0.000592,0.000356,0.000135,0.000191,0.015040,...,0.000501,0.000319,0.000266,0.000032,0.000077,0.001139,0.001244,0.000086,0.000083,0.000155,0.000541,0.000440,0.001178,0.001557,0.033702,0.000201,0.001335,0.000285,0.000842,0.004463,0.000789,0.014116,0.000487,0.000247,0.000384,0.000055,0.000140,0.005482,0.000346,0.018673,0.000433,0.000602,0.000475,0.009049,0.000056,0.000240,0.000201,0.000420,0.001052,0.000981
4,0382N0Y.wav,0.000008,0.000577,0.000124,0.000009,0.000008,0.000010,0.000007,0.000097,0.000006,0.000259,0.000031,0.000043,0.000090,0.000014,0.000158,0.000015,0.000009,0.000373,0.000028,0.000012,0.000055,0.000029,0.000005,0.000008,0.000030,0.000016,0.000074,0.000011,0.000007,0.000014,0.000009,0.000019,0.000010,0.000030,0.000014,0.000127,0.000007,0.000045

In [ ]:
test_ids["fn"] = 'audio_files/'+test_ids["fn"]
test_ids.head()
test_ids.to_csv('drive/My Drive/sub.csv', index=False)
test_ids.head()

,fn,maize streak virus,disease,okukkoola,muwogo,mpeke,mucungwa,greens,garden,mango,bulimi,obuwuka,ebikoola,obulimi,ebisoolisooli,kaamulali,eddagala,beans,omuyembe,leaf,kisaanyi,leaves,butterfly,okuzifuuyira,micungwa,ppaapaali,emboga,kikolo,harvest,olusuku,coffee,super grow,rice,ensujju,okulima,worm,ebbugga,onion,ensigo,plantation,...,ejjobyo,omulimi,okusimba,sweet potatoes,okufuuyira,farming instructor,nnasale beedi,passion fruit,ekitooke,ebisaanyi,ekyeya,enva endiirwa,emisiri,emiyembe,amatooke,ebiwuka,farm,ebinyebwa,amappapaali,ebimera,kassooli,harvesting,emmwanyi,akamonde,obumonde,cabbages,akasaanyi,spread,ebirime,drought,kasaanyi,suckers,insects,fertilizer,nakavundira,ekiwojjolo,akawuka,ddagala,ebiwojjolo,obutungulu
0,audio_files/00118N3.wav,0.001904,0.000555,0.000628,0.000492,0.000179,0.000970,0.002059,0.003454,0.000227,0.000111,0.008612,0.000211,0.000195,0.004386,0.000685,0.000634,0.001874,0.000264,0.001865,0.001436,0.000844,0.035215,0.002442,0.000706,0.003954,0.000457,0.000338,0.003942,0.000115,0.021425,0.000280,0.011231,0.000840,0.000032,0.000273,0.000160,0.000135,0.000332,0.001648,...,0.000104,0.000039,0.002311,0.000460,0.005301,0.002158,0.000727,0.001118,0.005031,0.004282,0.000431,0.000075,0.000425,0.000335,0.000529,0.002397,0.000145,0.002615,0.003811,0.000311,0.000677,0.000666,0.000083,0.001102,0.000435,0.001982,0.098408,0.000242,0.000079,0.002136,0.003859,0.283925,0.000980,0.006723,0.005748,0.000190,0.034194,0.005212,0.000295,0.000683
1,audio_files/00P0NMV.wav,0.000114,0.000036,0.000092,0.000340,0.000062,0.000641,0.000079,0.000340,0.000052,0.000276,0.000354,0.000152,0.000405,0.000221,0.000187,0.000215,0.000045,0.000066,0.000069,0.000187,0.000031,0.000110,0.000240,0.000268,0.000258,0.000354,0.000072,0.000071,0.000025,0.000042,0.000066,0.000037,0.000593,0.000116,0.000191,0.001137,0.000056,0.000118,0.000966,...,0.000081,0.000177,0.000326,0.000198,0.000261,0.000388,0.001220,0.000448,0.000084,0.000269,0.000038,0.000204,0.000160,0.000060,0.000288,0.000327,0.000021,0.000053,0.000424,0.000149,0.000085,0.000072,0.000269,0.000304,0.000298,0.000136,0.000200,0.000040,0.000052,0.000039,0.000560,0.000165,0.000093,0.000353,0.909589,0.001492,0.000653,0.000866,0.000317,0.000642
2,audio_files/01QEEZI.wav,0.000061,0.003933,0.000982,0.008510,0.000258,0.000217,0.000065,0.000440,0.000054,0.192283,0.000104,0.000099,0.007933,0.000068,0.001013,0.000083,0.000201,0.071119,0.001991,0.000108,0.002438,0.000226,0.000079,0.000153,0.000581,0.000849,0.000124,0.000105,0.000421,0.000293,0.000046,0.000147,0.000075,0.000205,0.001080,0.000858,0.000107,0.000360,0.001083,...,0.001004,0.003934,0.000139,0.000006,0.000227,0.000216,0.000143,0.000041,0.000163,0.000094,0.000111,0.003327,0.000966,0.032828,0.000262,0.000209,0.000677,0.006549,0.000575,0.031873,0.000203,0.000059,0.002329,0.000121,0.001207,0.000045,0.000188,0.000097,0.429000,0.004770,0.000208,0.000086,0.000050,0.000396,0.000089,0.000128,0.000208,0.000186,0.000471,0.001070
3,audio_files/037YAED.wav,0.000500,0.000516,0.000089,0.000221,0.008674,0.000172,0.000235,0.236752,0.000076,0.002010,0.000478,0.000140,0.006835,0.000518,0.000697,0.000239,0.000247,0.001286,0.001730,0.000208,0.001137,0.001939,0.000069,0.000225,0.000823,0.000273,0.000461,0.001124,0.017625,0.001720,0.000267,0.000230,0.000873,0.000201,0.000592,0.000356,0.000135,0.000191,0.015040,...,0.000501,0.000319,0.000266,0.000032,0.000077,0.001139,0.001244,0.000086,0.000083,0.000155,0.000541,0.000440,0.001178,0.001557,0.033702,0.000201,0.001335,0.000285,0.000842,0.004463,0.000789,0.014116,0.000487,0.000247,0.000384,0.000055,0.000140,0.005482,0.000346,0.018673,0.000433,0.000602,0.000475,0.009049,0.000056,0.000240,0.000201,0.000420,0.001052,0.000981
4,audio_files/0382N0Y.wav,0.000008,0.000577,0.000124,0.000009,0.000008,0.000010,0.000007,0.000097,0.000006,0.000259,0.000031,0.000043,0.000090,0.000014,0.000158,0.000015,0.000009,0.000373,0.000028,0.000012,0.000055,0.000029,0.000005,0.000008,0.000030,0.000016,0.000074,0.000011,0.000007,0.000014,0.000009,0.

In [ ]:
"""sub = pd.DataFrame()
sub["fn"] = test_ids["fn"]

#for i, label in enumerate(labels):
#  sub[label] = [t[i] for t in pred_0]

sub.to_csv('drive/My Drive/new_sub.csv', index=False)
sub.head()
sub.shape"""

(1017, 1)